<a href="https://colab.research.google.com/github/TechRoC/Image_Caption_Generator/blob/master/Model_Training_and_Evalate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras==2.3
!pip install tensorflow==2.0

In [ ]:
from os import listdir
from pickle import dump
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

225214464/225209952 [==============================] - 6s 0us/step


In [ ]:
def extract_features(directory):
	cont = 0
	model = InceptionResNetV2()
	model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
	print(model.summary())
	features = dict()
	for name in listdir(directory):
		cont += 1
		filename = directory + '/' + name
		image = load_img(filename, target_size=(299, 299))
		image = img_to_array(image)
		image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
		image = preprocess_input(image)
		feature = model.predict(image, verbose=0)
		image_id = name.split('.')[0]
		features[image_id] = feature
		print(cont)
		print('>%s' % name)
	return features
directory = "/content/drive/My Drive/Flickr8k_Dataset/Flicker8k_Dataset"
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
dump(features, open('image_features.pkl', 'wb'))


Streaming output truncated to the last 5000 lines.
5597
>3628059004_5c3529b120.jpg
5598
>1928319708_ccf1f4ee72.jpg
5599
>3564148252_aa4cb36a32.jpg
5600
>230016181_0c52b95304.jpg
5601
>2271264741_aa8f73f87c.jpg
5602
>2292406847_f366350600.jpg
5603
>490390951_87395fcb1c.jpg
5604
>1378557186_4bd1da6834.jpg
5605
>3337046794_296bd2c7e0.jpg
5606
>3251648670_9339943ba2.jpg
5607
>3368865171_597d51cdd5.jpg
5608
>3355756569_b430a29c2a.jpg
5609
>410422753_de506155fa.jpg
5610
>308014594_f1d5e75507.jpg
5611
>3364797223_1f0b2f98ed.jpg
5612
>299612419_b55fe32fea.jpg
5613
>2616561200_ea079f285a.jpg
5614
>2528489543_546c1ca81f.jpg
5615
>2596475173_58f11fc583.jpg
5616
>2038662925_f4fa8c2534.jpg
5617
>219843860_332e5ca7d4.jpg
5618
>2924483864_cfdb900a13.jpg
5619
>494907021_321e82877a.jpg
5620
>2110692070_8aaaa1ae39.jpg
5621
>3397803103_8a46d716f4.jpg
5622
>348380010_33bb0599ef.jpg
5623
>3122773470_b622205948.jpg
5624
>3286406057_a1668655af.jpg
5625
>179829865_095b040377.jpg
5626
>1561658940_a947f2446a.jp

In [ ]:
def load_doc(filename):
	file = open(filename, 'r')
	text = file.read()
	file.close()
	return text
 
filename = '/content/drive/My Drive/Flickr8k_text/Flickr8k.token.txt'
doc = load_doc(filename)

In [ ]:
def load_descriptions(doc):
	mapping = dict()
	for line in doc.split('\n'):
		tokens = line.split()
		if len(line) < 2:
			continue
		image_id, image_desc = tokens[0], tokens[1:]
		image_id = image_id.split('.')[0]
		image_desc = ' '.join(image_desc)
		if image_id not in mapping:
			mapping[image_id] = list()
		mapping[image_id].append(image_desc)
	return mapping
 

descriptions = load_descriptions(doc)

print('Loaded: %d ' % len(descriptions))

Loaded: 8092 


In [ ]:
import nltk
nltk.download('popular')
nltk.download('punkt')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [ ]:
def clean_descriptions(descriptions):
  import nltk
  from nltk.stem import WordNetLemmatizer
  from nltk.corpus import stopwords
  lemmatizer = WordNetLemmatizer()
  data = ['and','an','the','any','all','also','his','most','mostly','almost','about']
  for key, desc_list in descriptions.items():
    for i in range(len(desc_list)):
      desc = nltk.word_tokenize(desc_list[i])
      desc = [word.lower() for word in desc]
      desc = [lemmatizer.lemmatize(word) for word in desc if word not in set(data)]
			# remove hanging 's' and 'a'
      desc = [word for word in desc if len(word)>1]
			# remove tokens with numbers in them
      desc = [word for word in desc if word.isalpha()]
			# store as string
      desc_list[i] =  ' '.join(desc)
 
# clean descriptions
clean_descriptions(descriptions)

In [ ]:
def to_vocabulary(descriptions):
  i = 0
  all_desc = set()
  for key in descriptions.keys():
    [all_desc.update(d.split()) for d in descriptions[key]]
  return all_desc
 
# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))

Vocabulary Size: 7137


In [ ]:
def save_descriptions(descriptions, filename):
  lines = list()
  for key, desc_list in descriptions.items():
    for desc in desc_list:
      lines.append(key + ' ' + desc)
  data = '\n'.join(lines)
  print(data)
  file = open(filename, 'w')
  file.write(data)
  file.close()
 
# save descriptions
save_descriptions(descriptions, 'image_txt_descriptions.txt')

1000268201_693b08cb0e child in pink dress is climbing up set of stair in entry way
1000268201_693b08cb0e girl going into wooden building
1000268201_693b08cb0e little girl climbing into wooden playhouse
1000268201_693b08cb0e little girl climbing stair to her playhouse
1000268201_693b08cb0e little girl in pink dress going into wooden cabin
1001773457_577c3a7d70 black dog spotted dog are fighting
1001773457_577c3a7d70 black dog dog playing with each other on road
1001773457_577c3a7d70 black dog white dog with brown spot are staring at each other in street
1001773457_577c3a7d70 two dog of different breed looking at each other on road
1001773457_577c3a7d70 two dog on pavement moving toward each other
1002674143_1b742ab4b8 little girl covered in paint sits in front of painted rainbow with her hand in bowl
1002674143_1b742ab4b8 little girl is sitting in front of large painted rainbow
1002674143_1b742ab4b8 small girl in grass play with fingerpaint in front of white canvas with rainbow on it
10

In [ ]:
from numpy import array
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import *
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [ ]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions


In [ ]:
filename = '/content/drive/My Drive/Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('/content/drive/My Drive/image_txt_descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))

Dataset: 6000
Descriptions: train=6000


In [ ]:
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

In [ ]:
train_features = load_photo_features('/content/drive/My Drive/image_features.pkl', train)
print('Photos: train=%d' % len(train_features))

Photos: train=6000


In [ ]:
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

In [ ]:
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

Vocabulary Size: 6211
Description Length: 30


In [ ]:
def create_sequences(tokenizer, max_length, desc_list, photo, vocab_size):
	X1, X2, y = list(), list(), list()
	# walk through each description for the image
	for desc in desc_list:
		# encode the sequence
		seq = tokenizer.texts_to_sequences([desc])[0]
		# split one sequence into multiple X,y pairs
		for i in range(1, len(seq)):
			# split into input and output pair
			in_seq, out_seq = seq[:i], seq[i]
			# pad input sequence
			in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
			# encode output sequence
			out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
			# store
			X1.append(photo)
			X2.append(in_seq)
			y.append(out_seq)
	return array(X1), array(X2), array(y)

# define the captioning model
def data_generator(descriptions, photos, tokenizer, max_length, vocab_size):
	# loop for ever over images
	while 1:
		for key, desc_list in descriptions.items():
			# retrieve the photo feature
			photo = photos[key][0]
			in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo, vocab_size)
			yield [[in_img, in_seq], out_word]

In [ ]:
path_to_glove_file = '/content/drive/My Drive/Glove/glove.6B.300d.txt'

import numpy as np
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
hits = 0
misses = 0
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 5722 words (488 misses)


In [ ]:
from tensorflow import keras


In [ ]:
def define_model(vocab_size, max_length):
	# feature extractor model
	inputs1 = Input(shape=(1536,))
	fe1 = Dropout(0.5)(inputs1)
	fe2 = Dense(256, activation='relu')(fe1)
	# sequence model
	inputs2 = Input(shape=(max_length,))
	se1 = e = Embedding(vocab_size, 300, weights=[embedding_matrix],  mask_zero=True, trainable=False)(inputs2)
	se2 = Dropout(0.5)(se1)
	se3 = LSTM(256)(se2)
	# decoder model
	decoder1 = add([fe2, se3])
	decoder2 = Dense(256, activation='relu')(decoder1)
	outputs = Dense(vocab_size, activation='softmax')(decoder2)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	# compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam')
	# summarize model
	model.summary()
	plot_model(model, to_file='model.png', show_shapes=True)
	return model


In [ ]:
model = define_model(vocab_size, max_length)
# train the model, run epochs manually and save after each epoch
epochs = 20
steps = len(train_descriptions)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1536)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 30, 300)      1863300     input_4[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 1536)         0           input_3[0][0]                    
____________________________________________________________________________________________

In [ ]:
for i in range(epochs):
	# create the data generator
	generator = data_generator(train_descriptions, train_features, tokenizer, max_length, vocab_size)
	# fit for one epoch
	model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
	# save model
	model.save('/content/drive/My Drive/caption_model/model_' + str(i) + '.h5')

Epoch 1/1
6000/6000 [==============================] - 1563s 260ms/step - loss: 4.4730
Epoch 1/1
6000/6000 [==============================] - 1601s 267ms/step - loss: 3.7476
Epoch 1/1
6000/6000 [==============================] - 1500s 250ms/step - loss: 3.5182
Epoch 1/1
6000/6000 [==============================] - 1429s 238ms/step - loss: 3.3866
Epoch 1/1
6000/6000 [==============================] - 1429s 238ms/step - loss: 3.2950
Epoch 1/1
6000/6000 [==============================] - 1435s 239ms/step - loss: 3.2333
Epoch 1/1
6000/6000 [==============================] - 1430s 238ms/step - loss: 3.1817
Epoch 1/1
6000/6000 [==============================] - 1431s 239ms/step - loss: 3.1477
Epoch 1/1
6000/6000 [==============================] - 1432s 239ms/step - loss: 3.1205
Epoch 1/1
6000/6000 [==============================] - 1431s 239ms/step - loss: 3.0948
Epoch 1/1
6000/6000 [==============================] - 1427s 238ms/step - loss: 3.0770
Epoch 1/1
6000/6000 [======================

In [ ]:
from numpy import argmax
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r') 
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text

# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc_list in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		references = [d.split() for d in desc_list]
		actual.append(references)
		predicted.append(yhat.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

# prepare tokenizer on train set

# load training dataset (6K)
filename = '/content/drive/My Drive/Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('/content/drive/My Drive/image_txt_descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

# prepare test set

# load test set
filename = '/content/drive/My Drive/Flickr8k_text/Flickr_8k.testImages.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('/content/drive/My Drive/image_txt_descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('/content/drive/My Drive/image_features.pkl', test)
print('Photos: test=%d' % len(test_features))

# load the model
filename = '/content/drive/My Drive/caption_model/model_17.h5'
model = load_model(filename)
# evaluate model
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

Dataset: 6000
Descriptions: train=6000
Vocabulary Size: 6211
Description Length: 30
Dataset: 1000
Descriptions: test=1000
Photos: test=1000
BLEU-1: 0.547434
BLEU-2: 0.300629
BLEU-3: 0.200851
BLEU-4: 0.089461


In [ ]:
from pickle import dump
dump(tokenizer, open('tokenizer.pkl', 'wb'))